# Student Admission

In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
data = pd.read_csv('https://stats.idre.ucla.edu/stat/data/binary.csv')

In [4]:
print(data.head())

   admit  gre   gpa  rank
0      0  380  3.61     3
1      1  660  3.67     3
2      1  800  4.00     1
3      1  640  3.19     4
4      0  520  2.93     4


In [15]:
X = data.iloc[:,[1,2,3]]
y = data.iloc[:,[0]]
X, y

(     gre   gpa  rank
 0    380  3.61     3
 1    660  3.67     3
 2    800  4.00     1
 3    640  3.19     4
 4    520  2.93     4
 ..   ...   ...   ...
 395  620  4.00     2
 396  560  3.04     3
 397  460  2.63     2
 398  700  3.65     2
 399  600  3.89     3
 
 [400 rows x 3 columns],      admit
 0        0
 1        1
 2        1
 3        1
 4        0
 ..     ...
 395      0
 396      0
 397      0
 398      0
 399      0
 
 [400 rows x 1 columns])

In [19]:
X_train, y_train, X_test, y_test = train_test_split(X,y, test_size=0.3, random_state=42)
X_train.head()

,gre,gpa,rank
157,480,3.58,1
109,480,3.45,2
17,360,2.56,3
347,580,3.80,2
24,760,3.35,2
